MCS - BeNeFri
<br/>Social Media Analytics, Spring 2020
<br/>Project - Community Detection in a Facebook network
<br/>@authors: Guillaume Chacun, Fabien Vorpe

# Community Detection in a Facebook network

## Dataset

### Load the graph

In [1]:
import networkx as nx
from tqdm import tqdm

In [2]:
edges_filename = 'example.txt' #facebook_combined

In [3]:
# Load the Facebook dataset
graph = nx.read_edgelist(f'data/{edges_filename}')
#graph = nx.gnp_random_graph(100, 0.2)

edge_list = ['%i %i {\'weight\': %i}' % (int(d[0]), int(d[1]), int(d[2])) for d in list(graph.edges.data('weight', default=1))]
graph = nx.parse_edgelist(edge_list)

### Explore the properties of the graph

In [4]:
# print number of nodes / edges
print('Number of nodes:', graph.number_of_nodes())
print('Number of edges:', graph.number_of_edges())

Number of nodes: 16
Number of edges: 28


In [5]:
import statistics
print('Average shortest path length:', nx.average_shortest_path_length(graph))
print('Average degree:', statistics.mean([graph.degree(node) for node in graph.nodes()]))
print('Radius:', nx.radius(graph))
print('Diameter:', nx.diameter(graph))
print('Density:', nx.density(graph))
#print('Eccentricity:', nx.eccentricity(graph))
#print('Center:', nx.center(graph))
#print('Periphery:', nx.periphery(graph))

Average shortest path length: 2.308333333333333
Average degree: 3.5
Radius: 3
Diameter: 5
Density: 0.23333333333333334


## Algorithms implementation

### Louvain

Implement the Louvain method for community detection.  

In [6]:
def louvain_passage(graph):
    """
    Performs one passage of Louvain's algorithm to detect communities in a graph.
    :param graph: networkx graph
    :returns: dictionary with graph's nodes as keys and communities' id as values
    """
    m = sum([edge[2]['weight'] for edge in graph.edges().data()])
    prev_communities = {}
    communities = {i: i for i in graph.nodes} # keys: nodes' id, values: nodes' community id
    while list(prev_communities.values()) != list(communities.values()): # while communities are changing
        prev_communities = communities.copy()
        
        communities = {k: v for k, v in sorted(communities.items(), 
                                               key=lambda item: int(item[1]))} # sort by node id (keys)
        for nodei in tqdm(communities.keys()):
            communityi = communities[nodei]
            di = sum([graph.get_edge_data(nodei, neighbor)['weight']
                      for neighbor in graph.neighbors(nodei)])
            # can't use degree() as we're working with undirected graphs and degree() counts self loops twice
            
            neighbor_communities = set(communities[neighbor] for neighbor in graph.neighbors(nodei))
            
            # compute modularity gain for each neighbor community of nodei
            modularity_gains = {} # keys: community' id, values: modularity gain
            
            nodes_of_each_community = {}
            for node, community in communities.items():
                try:
                    nodes_of_each_community[community].append(node)
                except:
                    nodes_of_each_community[community] = [node]
                    
            for communityj in neighbor_communities:
                #nodes_in_communityj = [node for node, community in communities.items() 
                #                       if community == communityj] # modified
                nodes_in_communityj = nodes_of_each_community[communityj] # get nodes belonging to community j
                
                dj = 0
                links_ij = []
                for node in nodes_in_communityj:
                    dj += sum([graph.get_edge_data(node, neighbor)['weight'] 
                               for neighbor in graph.neighbors(node) 
                               if node != nodei])
                    links_ij.append(graph.get_edge_data(nodei, node)['weight'] if node in graph.neighbors(nodei) else 0)
                
                #dj2 = sum([graph.get_edge_data(node, neighbor)['weight'] 
                #          for node in nodes_in_communityj
                #          for neighbor in graph.neighbors(node) 
                #          if node != nodei]) # modified
                
                #links_ij = [graph.get_edge_data(nodei, node)['weight'] if node in graph.neighbors(nodei) else 0 
                #            for node in nodes_in_communityj] # modified
                
                # compute modularity
                modularity_gains[communityj] = 1 / (2 * m) * (dij - (di * dj / m))
                
            # update node's community
            modularity_gains = {k: v 
                                for k, v in sorted(modularity_gains.items(), 
                                                   key=lambda item: item[1])} # sort by modularity gain (values)
            communities[nodei] = list(modularity_gains.keys())[-1]
    return communities

In [7]:
def louvain_create_hypernodes(graph, communities):
    """
    Creates the hypernodes graph after one passage of Louvain's aglorithm.
    :param graph: networkx graph
    :param communities: dictionary with graph's nodes as keys and communityies' id as values
    :returns: new networkx graph (communities became hypernodes)
    """
    new_graph = nx.Graph()
    # create hypernodes and reflective edges
    for community in set(communities.values()):
        new_graph.add_node(community)

        # count number of edges in community
        nodes_in_community = [n for n, c in communities.items() if community == c]
        links_intra_community = [1 if neighbor in nodes_in_community else 0 
                                 for node in nodes_in_community
                                 for neighbor in graph.neighbors(node)]
        new_graph.add_edge(community, community, weight=sum(links_intra_community)/2)

    # create inter communities edges
    for community in new_graph.nodes:
        nodes_in_community = [n for n, c in communities.items() if community == c]
        neighbors_of_community = [neighbor
                                  for node in nodes_in_community
                                  for neighbor in graph.neighbors(node) 
                                  if communities[neighbor] != community]
        for other_community in set(communities.values()):
            if other_community != community:
                nodes_in_other_community = [n for n, c in communities.items() if other_community == c]
                nodes_neighbors_in_other_community = [node for node in neighbors_of_community 
                                                      if node in nodes_in_other_community]
                edge_weight = len(nodes_neighbors_in_other_community)
                if other_community not in new_graph.neighbors(community) and edge_weight > 0:
                    new_graph.add_edge(community, other_community, weight=edge_weight)
    return new_graph

In [8]:
def louvain(graph):
    """
    Louvain's algorithm to detect communities in a given graph.
    :param graph: networkx graph
    :returns: graph of communities, a dictionary with hypernodes as keys and communities' id as values and 
              a dictionary with graph's nodes as keys and communities' id as values
    """
    all_communities_hypernodes = []
    all_communities_nodes = [{node: node for node in list(graph.nodes())}]
    communities_id = set(graph.nodes())
    passage_id = 0
    while True: # passage
        communities = louvain_passage(graph)
        all_communities_hypernodes.append(communities)

        # check if modif
        if communities_id == set(communities.values()):
            break
        communities_id = set(communities.values())
        
        new_passage_communities = {node: all_communities_hypernodes[passage_id][all_communities_nodes[passage_id][node]] 
                                   for node in all_communities_nodes[passage_id].keys()}
        all_communities_nodes.append(new_passage_communities)
        
        graph = louvain_create_hypernodes(graph, communities)
        
        passage_id += 1
        print('Graph after passage', passage_id)
        draw_graph(graph)
        
    
    return graph, all_communities_hypernodes, all_communities_nodes[1:]

### Clustering coefficient
Implement the clustering coefficient for a graph.

In [9]:
def average_clustering_coefficient(graph):
    """
    Computes the average clustering coefficient for a given graph.
    :param graph: networkx graph
    :returns: the average clustering coefficient of the given graph
    """
    return statistics.mean(clustering_coefficient(graph).values())

In [10]:
def clustering_coefficient(graph, top_k=None):
    """
    Computes the clustering coefficient for each node in a given graph.
    :param graph: networkx graph
    :param top_k: top k nodes with highest clustering coefficient to return. If None returns all users.
    :returns: dictionary with graph's nodes as keys and their clustering coefficient as values
    """
    coeffs = {}
    for v in graph.nodes(): # compute coeff for each node
        neighbors = list(graph.neighbors(v))
        
        connected_pairs_neighbors = []
        for n1 in neighbors: # for each neighbor n1 of node v
            for n2 in graph.neighbors(n1): # for each neighbor n2 of neighbor n1 of node v
                if n2 in neighbors: # if n2 is also a neighbor of node v: (n1,n2) is a connected pair of v's neighbors
                    connected_pairs_neighbors.append((n1, n2))
        # C(v) = #connected pairs of v's neighbors / #pairs of v's neighbors
        nb_pairs_neighnbors = (len(neighbors) * (len(neighbors)-1))
        coeffs[v] = len(connected_pairs_neighbors) / nb_pairs_neighnbors if nb_pairs_neighnbors > 0 else 0
        
    coeffs = {k: v 
              for k, v in sorted(coeffs.items(), 
                                 key=lambda item: item[1], 
                                 reverse=True)} # order by coeffs (values)
    return dict(list(coeffs.items())[:top_k if top_k else -1])

### Random walk

In [11]:
def percentage_visited_communities(visited_nodes, communities):
    """
    Computes the percentages of visited communities from the given list of visited nodes.
    :param visited_nodes: list of visited nodes
    :param communities: dictionary with nodes as keys and communities as values
    :returns: percentage of visited communities
    """
    return len(set(communities[node] for node in visited_nodes)) / len(set(communities.values())) * 100

In [12]:
def random_walk(graph, starting_node, steps, random_walk_strategy):
    """
    Visits a graph with a given random walk strategy.
    :param graph: networkx graph
    :param starting_node: node of the given graph which serves as the starting point of the random walk
    :param steps: number of steps to perform
    :param random_walk_strategy: function receiving a graph and a node and returning a node's neighbor to visit
    :returns: list of nodes in their order of visit
    """
    path = [starting_node]
    node = path[0]
    for _ in range(steps):
        node = random_walk_strategy(graph, node)
        path.append(node)
    return path

In [13]:
from random import randrange, choices

In [14]:
def rw_strategy_degree(graph, node):
    """
    Random walk strategy - the degree of a neighbor influences the probability to move to this neighbor.
    :param graph: networkx graph
    :param node: node of the given graph from which we search a neighbor to move to
    :returns: node's neighbor to move to
    """
    neighbors = list(graph.neighbors(node))
    probs = [graph.degree(neighbor) for neighbor in neighbors]
    probs = [p/sum(probs) for p in probs]
    return choices(list(neighbors), probs)[0]

In [15]:
def rw_strategy_random(graph, node):
    """
    Random walk strategy - completely random.
    :param graph: networkx graph
    :param node: node of the given graph from which we search a neighbor to move to
    :returns: node's neighbor to move to
    """
    neighbors = list(graph.neighbors(node))
    rdm_idx = randrange(len(neighbors)-1)
    return neighbors[rdm_idx]

In [16]:
def rw_strategy_connected_neighbors(graph, node):
    """
    Random walk strategy - the inverse number of connected neighbors influences the probability to move to this neighbor.
    :param graph: networkx graph
    :param node: node of the given graph from which we search a neighbor to move to
    :returns: node's neighbor to move to
    """
    neighbors = list(graph.neighbors(node))
    probs = [len([neighbor_of_neighbor 
                  for neighbor_of_neighbor in list(graph.neighbors(neighbor)) 
                  if neighbor_of_neighbor in neighbors]) 
             for neighbor in neighbors]
    probs = [sum(probs)-p for p in probs]
    probs = [p/sum(probs) if sum(probs) > 0 else 1/len(probs) for p in probs]
    return choices(list(neighbors), probs)[0]

### Utils

In [17]:
import matplotlib.pyplot as plt
def draw_graph(graph):
    print(graph.edges.data())
    # visualize network
    options = {
        'node_color': (0,0,1,.6),
        'node_size': 30,
        'line_color': (.8,.8,.8,1),
        'linewidths': 0,
        'width': .1,
        'with_labels': 'True',
    }
    nx.draw_networkx(graph, **options)
    plt.show()

## Analysis

In [18]:
# Identify users’ communities in the Facebook network using Louvain
new_graph, communities_hypernodes, communities_nodes = louvain(graph)

  0%|                                                                                           | 0/16 [00:00<?, ?it/s]


NameError: name 'dij' is not defined

In [ ]:
print(f'There are {len(set(communities_nodes[-1].values()))} communities.')
print(communities_nodes[-1])

In [ ]:
import community
louvain_communities = community.best_partition(graph)
nb_louvain_communities = len(set(louvain_communities.values()))
louvain_communities = {k: v for k, v in sorted(louvain_communities.items(), key=lambda item: item[0])} # sort by key (node id)
print(f'There are {nb_louvain_communities} communities according to Louvain\' method.')

dendo = community.generate_dendrogram(graph)
for level in range(len(dendo)) :
    print("partition at level", level, "is", community.partition_at_level(dendo, level))

In [ ]:
# Identify the top k users with the highest clustering coefficient in the graph.
# Experiment with different values of k and choose the most appropriate one.
print('Average clustering coefficient:', average_clustering_coefficient(graph))
print('Top k=3 users with highest clustering coefficients:', clustering_coefficient(graph, top_k=3))

In [ ]:
nx.clustering(graph)

In [ ]:
# Evaluate different random walk strategies to spread a message across the network.
# The message should reach as many different communities as possible.
starting_node = list(graph.nodes())[0]
steps = 10
rw_result_strategy_degree = random_walk(graph, starting_node, steps, rw_strategy_degree)
rw_result_strategy_random = random_walk(graph, starting_node, steps, rw_strategy_random)
rw_result_stategy_connected_neighbors = random_walk(graph, starting_node, steps, rw_strategy_connected_neighbors)

print('Percentage of visited communities with the degree based rw strategy:', percentage_visited_communities(rw_result_strategy_degree, communities_nodes[-1]), '%')
print('Percentage of visited communities with full random rw strategy:', percentage_visited_communities(rw_result_strategy_random, communities_nodes[-1]), '%')
print('Percentage of visited communities with the connected neighbors rw strategy:' ,percentage_visited_communities(rw_result_stategy_connected_neighbors, communities_nodes[-1]), '%')

## Visualization

In [ ]:
def draw_communities(graph, communities, pos_param=None):
    nodes_community = [int(community) for community in communities.values()]
    if pos_param:
        nx.draw(graph, cmap=plt.get_cmap('viridis'), node_color=nodes_community, with_labels=True, font_color='white', pos=pos_param)
    else:
        nx.draw(graph, cmap=plt.get_cmap('viridis'), node_color=nodes_community, with_labels=True, font_color='white')
    plt.show()

In [ ]:
def draw_path(graph, path, pos_param=None):
    edges_path = []
    for i in range(len(path)-1):
        edges_path.append((path[i], path[i+1]))
    pos = nx.spring_layout(graph)
    if pos_param:
        pos = pos_param
    nx.draw(graph, pos=pos, with_labels=True)
    nx.draw_networkx_nodes(graph, pos, nodelist=path[1:], node_color='r')
    nx.draw_networkx_nodes(graph, pos, nodelist=[path[0]], node_color='g')
    nx.draw_networkx_edges(graph, pos, edgelist=edges_path, width=2.0, edge_color='r')

In [ ]:
pos = nx.spring_layout(graph)

# Visualize the output of Louvain by coloring the nodes according to their assigned Communities.
draw_communities(graph, communities_nodes[-1], pos)

# Visualize the output of applying Random Walk algorithm, by highlighting the sequence of nodes selected in a path.
draw_path(graph, rw_result_strategy_degree, pos)